In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import googletrans
from googletrans import Translator
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
import folium

translator = Translator()

plt.rc("font", family="AppleGothic")
plt.rc("axes", unicode_minus=False)

set_matplotlib_formats("retina")

df1 = pd.read_csv("../../2021_elementary.csv")
df2 = pd.read_csv("../../2021_juniorHigh.csv")
df3 = pd.read_csv("../../2021_High.csv")

df = pd.concat([df1,df2,df3])

df.rename(columns=lambda x: translator.translate(x).text, inplace=True)

df.head()

,Attempt Office of Education,Local school administration,area,Information Disclosure School Code,School name,School Code,Establishment,School Characteristics,Whether,Established type,...,Zip code,School Road Name Postal Code,School Road Name Address,School Road Name Detail Address,Latitude,Hardness,Phone number,Fax number,Home Address,Men's engineering classification
0,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 서초구,S000003511,서울교육대학교부설초등학교,2,국립,NaN,N,부설,...,137742,6639.0,서울특별시 서초구 서초중앙로 96,"(서초동,서울교대초등학교)",37.490739,127.015424,02-6009-6000,02-6009-6090,http://ps.snue.ac.kr/,남녀공학
1,서울특별시교육청,서울특별시중부교육지원청,서울특별시 종로구,S000003563,서울대학교사범대학부설초등학교,2,국립,NaN,N,부설,...,110810,3087.0,서울특별시 종로구 대학로 64,"(동숭동,서울사대부설초등학교)",37.577017,127.003091,02-768-1500,02-766-4085,http://www.snu.es.kr,남녀공학
2,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 강남구,S010000738,서울개일초등학교,2,공립,NaN,N,단설,...,135804,6323.0,서울특별시 강남구 개포로 401,", 개일초등학교 (개포동,개일초등학교)",37.486214,127.057742,02-571-8320,02-571-8325,http://gaeil.sen.es.kr,남녀공학
3,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 강남구,S010000741,서울구룡초등학교,2,공립,NaN,N,단설,...,135244,6305.0,서울특별시 강남구 개포로 263,"(개포동,구룡초등학교)",37.481193,127.051755,02-573-5017,02-575-4237,http://kuryong.sen.es.kr,남녀공학
4,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 강남구,S010000742,서울논현초등학교,2,공립,NaN,N,단설,...,135010,6119.0,서울특별시 강남구 강남대로120길 33,"(논현동,논현초등학교)",37.500000,127.000000,02-514-6636,02-514-7072,http://nonhyun.sen.es.kr,남녀공학


In [5]:
seoul_map = folium.Map(location=[37.5614378,126.9751701], zoom_start=14)
folium.Marker([37.5614378,126.9751701], popup='City Hall').add_to(seoul_map)

df = df[['Information Disclosure School Code','School name','Latitude','Hardness']].copy()
school_df = df.drop_duplicates()

In [6]:
def graduate_preprocrssing(path):
    raw_graduate = pd.read_excel(path, sheet_name='2018_졸업생의 진로 현황(중)')
    select_col = raw_graduate[['지역', '학교명','정보공시 \n 학교코드', '졸업자.2', '(특수목적고)과학고 진학자.2','(특수목적고)외고ㆍ국제고 진학자.2']]
    select_col.columns = ['지역', '학교명', '학교코드', '졸업자', '과고', '외고']
    graduate_data = select_col.drop(0)
    graduate_data['과고'] = pd.to_numeric(graduate_data['과고'])
    graduate_data['외고'] =  pd.to_numeric(graduate_data['외고']) 
    graduate_data['졸업자'] =  pd.to_numeric(graduate_data['졸업자']) 
    graduate_data['총합'] = graduate_data['과고'] + graduate_data['외고']
    
    def get_sido(x):
        temp = x.split(' ')[0]
        if len(temp) != 4:
            return temp[:2]
        else:
            return temp[0] + temp[2]
    
    graduate_data['시도'] = graduate_data['지역'].dropna().apply(get_sido)
    graduate_data['구군'] = graduate_data['지역'].dropna().apply(lambda x: x.split(' ')[1])
    
    graduate_data.at[588,'시도'] = '부산'
    graduate_data.at[588,'구군'] = '기장군'
    graduate_data.at[3011,'시도'] = '경북'
    graduate_data.at[3011,'구군'] = '예천군'
    
    return graduate_data

graduate_path = "../../2018_graduates.xlsx"
gradu_df = graduate_preprocrssing(graduate_path)

/opt/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
gradu_df.head()

,지역,학교명,학교코드,졸업자,과고,외고,총합,시도,구군
1,서울특별시 성북구,서울대학교사범대학부설중학교,S000003514,234.0,4.0,5.0,9.0,서울,성북구
2,서울특별시 종로구,서울대학교사범대학부설여자중학교,S000003562,172.0,0.0,8.0,8.0,서울,종로구
3,서울특별시 강남구,개원중학교,S010000698,214.0,2.0,9.0,11.0,서울,강남구
4,서울특별시 서초구,경원중학교,S010000700,310.0,5.0,4.0,9.0,서울,서초구
5,서울특별시 강남구,구룡중학교,S010000701,256.0,3.0,4.0,7.0,서울,강남구


In [8]:
total_school_df = pd.merge(gradu_df, school_df, how='inner', left_on='학교코드', right_on='Information Disclosure School Code')

In [9]:
total_school_df.head()

,지역,학교명,학교코드,졸업자,과고,외고,총합,시도,구군,Information Disclosure School Code,School name,Latitude,Hardness
0,서울특별시 성북구,서울대학교사범대학부설중학교,S000003514,234.0,4.0,5.0,9.0,서울,성북구,S000003514,서울대학교사범대학부설중학교,37.596201,127.039164
1,서울특별시 종로구,서울대학교사범대학부설여자중학교,S000003562,172.0,0.0,8.0,8.0,서울,종로구,S000003562,서울대학교사범대학부설여자중학교,37.577928,127.003899
2,서울특별시 강남구,개원중학교,S010000698,214.0,2.0,9.0,11.0,서울,강남구,S010000698,개원중학교,37.491641,127.071415
3,서울특별시 서초구,경원중학교,S010000700,310.0,5.0,4.0,9.0,서울,서초구,S010000700,경원중학교,37.510997,127.008879
4,서울특별시 강남구,구룡중학교,S010000701,256.0,3.0,4.0,7.0,서울,강남구,S010000701,구룡중학교,37.486153,127.056123


In [10]:
seoul_school = total_school_df[total_school_df['시도'] == '서울']
good_school = seoul_school[seoul_school['총합'] > 0]
seoul_map = folium.Map(location=[37.5614378,126.9751701], zoom_start=11)

for n in good_school.index:
    folium.Marker([ good_school['Latitude'][n], good_school['Hardness'][n] ], popup=good_school['School name'][n]).add_to(seoul_map)

In [11]:
seoul_school['비율'] = seoul_school['총합'] /seoul_school['졸업자'] * 100
good_school = seoul_school[seoul_school['비율'] >= 3]
bad_school = seoul_school[seoul_school['비율'] < 3]
seoul_map = folium.Map(location=[37.5614378,126.9751701], zoom_start=11)

for n in good_school.index:
    folium.CircleMarker([ good_school['Latitude'][n], good_school['Hardness'][n] ], color='blue',fill_color='blue', radius=7).add_to(seoul_map)

for n in bad_school.index:
    folium.CircleMarker([ bad_school['Latitude'][n], bad_school['Hardness'][n] ], color='brown',fill_color='brown',  radius=7).add_to(seoul_map)

<ipython-input-11-2e62b9713907>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_school['비율'] = seoul_school['총합'] /seoul_school['졸업자'] * 100


In [12]:
subway_df = pd.read_csv('../../subway_station.csv',encoding='cp949')
for n in subway_df.index:
    folium.CircleMarker([ subway_df['역위도'][n], subway_df['역경도'][n] ], color='green',fill_color='green', radius=20).add_to(seoul_map)


In [13]:
seoul_map